In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# load dataframes

In [3]:
character_file = r'Dataset\MovieSummaries\character.metadata.tsv'
movie_file = r"Dataset\MovieSummaries\movie.metadata.tsv"
name_clusters_file = r'Dataset\MovieSummaries\name.clusters.txt' 
summaries_file = r'Dataset\MovieSummaries\plot_summaries.txt'
tvtropes_clusters_file = r'Dataset\MovieSummaries\tvtropes.clusters.txt'

## Character dataframe

In [4]:
column_names = [
    "wiki_movie_id",
    "freebase_movie_id",
    "release_date",
    "character_name",
    "actor_birth_date",
    "actor_gender",
    "actor_height_m",
    "actor_ethnicity_id",
    "actor_name",
    "actor_age_at_release",
    "character_actor_map_id",
    "character_id",
    "actor_id"
]
character = pd.read_csv(character_file, delimiter='\t', names = column_names)
character.head()

,wiki_movie_id,freebase_movie_id,release_date,character_name,actor_birth_date,actor_gender,actor_height_m,actor_ethnicity_id,actor_name,actor_age_at_release,character_actor_map_id,character_id,actor_id
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


## Movie dataframe

In [5]:
column_names = ['Wikipedia_movie_ID',
                'Freebase_movie_ID',
                'Movie_name',
                'Movie_release_date',
                'Movie_box_office_revenu', 'Movie_runtime', 'Movie_languages', 'Movie_countries', 'Movie_genres']
movie = pd.read_csv(movie_file, delimiter='\t', names = column_names)
movie['Movie_release_date_datetime']= pd.to_datetime(movie['Movie_release_date'], format='mixed', errors='coerce')
movie['Movie_release_year'] = movie['Movie_release_date_datetime'].dt.strftime('%Y')
movie.head()

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenu,Movie_runtime,Movie_languages,Movie_countries,Movie_genres,Movie_release_date_datetime,Movie_release_year
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",2001-08-24,2001
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",2000-02-16,2000
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",1988-01-01,1988
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",1987-01-01,1987
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",1983-01-01,1983


In [7]:
movie['number_genres'] = movie['Movie_genres'].str.split(',',expand=True).count(axis='columns')
movie_genre = movie[['Wikipedia_movie_ID','number_genres']]
nb_movies = movie_genre['Wikipedia_movie_ID'].unique().size
genres = movie['Movie_genres'].str.split(r'{|}',expand=True).fillna(0)
genres = genres[1].str.split(',',expand=True).fillna(0)
L=[]

for i in range(nb_movies):
    L+=[[movie_genre['Wikipedia_movie_ID'][i]]]*movie_genre['number_genres'][i]

j=0
k=0
for i in range(nb_movies):
    while j<17 and genres[j][i]!=0: #get rid of 17
        L[k+j]=L[k+j]+[genres[j][i]]
        j+=1
    k=k+j
    j=0


df_genres = pd.DataFrame(L,columns=['Wikipedia_movie_ID','Movie_genres'])
df_genres2 = df_genres["Movie_genres"].str.split(r":",expand=True)
df_genres = df_genres.merge(df_genres2,left_index=True,right_index=True)

df_genres = df_genres.drop(columns=["Movie_genres"]).rename(columns={0: "Movie_genres_ID", 1: "Movie_genres"})
df_genres.columns
df_genres.head(30)


,Wikipedia_movie_ID,Movie_genres_ID,Movie_genres
0,975900,"""/m/01jfsb""","""Thriller"""
1,975900,"""/m/06n90""","""Science Fiction"""
2,975900,"""/m/03npn""","""Horror"""
3,975900,"""/m/03k9fj""","""Adventure"""
4,975900,"""/m/0fdjb""","""Supernatural"""
5,975900,"""/m/02kdv5l""","""Action"""
6,975900,"""/m/09zvmj""","""Space western"""
7,3196793,"""/m/02n4kr""","""Mystery"""
8,3196793,"""/m/03bxz7""","""Biographical film"""
9,3196793,"""/m/07s9rl0""","""Drama"""


In [22]:

df_genres['Movie_genres'].unique()

array([' "Thriller"', ' "Science Fiction"', ' "Horror"', ' "Adventure"',
       ' "Supernatural"', ' "Action"', ' "Space western"', ' "Mystery"',
       ' "Biographical film"', ' "Drama"', ' "Crime Drama"',
       ' "Crime Fiction"', ' "Erotic thriller"',
       ' "Psychological thriller"', ' "Short Film"', ' "Silent film"',
       ' "Indie"', ' "Black-and-white"', ' "Comedy"', ' "Family Film"',
       ' "Fantasy"', ' "World cinema"', ' "Musical"',
       ' "Japanese Movies"', ' "Action/Adventure"', ' "Romantic comedy"',
       ' "Ensemble Film"', ' "Comedy-drama"', ' "Romantic drama"',
       ' "Romance Film"', ' "Costume drama"', ' "War film"', ' "Epic"',
       ' "Period piece"', ' "Film adaptation"', ' "Animation"',
       ' "Children\'s/Family"', ' "Children\'s Fantasy"', None,
       ' "Future noir"', ' "Comedy film"', ' "Coming of age"',
       ' "Suspense"', ' "Crime Thriller"', ' "Black comedy"',
       ' "Bollywood"', ' "Marriage Drama"', ' "Wuxia"',
       ' "Martial Arts Fi

In [15]:
L

[[975900, '"/m/01jfsb": "Thriller"'],
 [975900, ' "/m/06n90": "Science Fiction'],
 [975900, ' "/m/03npn": "Horror'],
 [975900, ' "/m/03k9fj": "Adventure'],
 [975900, ' "/m/0fdjb": "Supernatural'],
 [975900, ' "/m/02kdv5l": "Action'],
 [975900, ' "/m/09zvmj": "Space western"'],
 [3196793, '"/m/02n4kr": "Mystery"'],
 [3196793, ' "/m/03bxz7": "Biographical film'],
 [3196793, ' "/m/07s9rl0": "Drama'],
 [3196793, ' "/m/0hj3n01": "Crime Drama"'],
 [28463795, '"/m/0lsxr": "Crime Fiction"'],
 [28463795, ' "/m/07s9rl0": "Drama"'],
 [9363483, '"/m/01jfsb": "Thriller"'],
 [9363483, ' "/m/0glj9q": "Erotic thriller'],
 [9363483, ' "/m/09blyk": "Psychological thriller"'],
 [261236, '"/m/07s9rl0": "Drama"}'],
 [13696889, '"/m/02hmvc": "Short Film"'],
 [13696889, ' "/m/06ppq": "Silent film'],
 [13696889, ' "/m/0219x_": "Indie'],
 [13696889, ' "/m/01g6gs": "Black-and-white'],
 [13696889, ' "/m/01z4y": "Comedy"'],
 [18998739, '"/m/0hqxf": "Family Film"'],
 [18998739, ' "/m/01hmnh": "Fantasy'],
 [1899873

In [78]:
a=[movie_genre['Wikipedia_movie_ID'][0]]*movie_genre['number_genres'][0]

In [79]:
a

[975900, 975900, 975900, 975900, 975900, 975900, 975900]

In [80]:
a = a + [movie_genre['Wikipedia_movie_ID'][1]]*movie_genre['number_genres'][1]

In [81]:
a

[975900,
 975900,
 975900,
 975900,
 975900,
 975900,
 975900,
 3196793,
 3196793,
 3196793,
 3196793]